In [1]:
#!pip install pandas datatable rpy2 matplotlib glmnet_py

In [2]:
!pip install glmnet_py

In [5]:
import rpy2.robjects as robjects
import numpy as np
from rpy2.robjects import pandas2ri
import pandas as pd
pandas2ri.activate()
from glmnet_py import glmnet

In [6]:
faces = pd.read_csv('faces_smaller_chunk.csv', index_col=0)

In [7]:
faces.head()

,userid,gender,age,country,facial_hair,pol,pol_dat_us,pol_dat_ca,pol_dat_uk,pol_fb_us,...,left_eye_status.no_glass_eye_open,left_eye_status.normal_glass_eye_close,left_eye_status.dark_glasses,right_eye_status.normal_glass_eye_open,right_eye_status.no_glass_eye_close,right_eye_status.occlusion,right_eye_status.no_glass_eye_open,right_eye_status.normal_glass_eye_close,right_eye_status.dark_glasses,ethnicity.value
1,1,0.0,NaN,united states,0.001713,liberal,NaN,NaN,NaN,0.0,...,1.8,0.0,0.0,0.0,0.0,99.9,0.1,0.0,0.0,black
2,2,1.0,NaN,united states,0.005151,liberal,NaN,NaN,NaN,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,black
3,3,0.0,NaN,united states,0.977098,conservative,NaN,NaN,NaN,1.0,...,0.3,0.0,15.5,39.7,0.0,0.0,0.0,0.0,60.2,india
4,4,1.0,24.36,united states,0.000478,liberal,NaN,NaN,NaN,0.0,...,98.9,0.0,0.5,0.0,0.0,10.2,81.7,0.0,8.1,black
5,5,1.0,NaN,united states,0.005617,liberal,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,2.3,0.0,97.3,0.1,0.1,0.2,asian


In [8]:
ground_truth = ["pol_dat_us", "pol_dat_ca","pol_dat_uk","pol_fb_us"]

In [9]:
cv_folds = 4

In [10]:
folds = np.random.randint(1,cv_folds+1, len(faces))

In [11]:
vgg = pd.read_csv('./vgg_chunks/chunk_1.csv', index_col=0, header=0);

In [12]:
vgg.shape

(10859, 2048)

In [13]:
col = f'pred_{ground_truth[0]}'
col

'pred_pol_dat_us'

In [14]:
faces[col] = np.nan

In [15]:
faces[col]

1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
         ..
10855   NaN
10856   NaN
10857   NaN
10858   NaN
10859   NaN
Name: pred_pol_dat_us, Length: 10859, dtype: float64

In [20]:
i = ground_truth[0]

In [23]:
not_na = faces[faces[i].isna()].index

test_index = not_na[folds[not_na] == 1]

train_index = not_na[folds[not_na] != 1]

In [22]:
faces[i][train_index]

2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
         ..
10848   NaN
10850   NaN
10851   NaN
10852   NaN
10853   NaN
Name: pol_dat_us, Length: 7201, dtype: float64

In [27]:
vgg.iloc[train_index]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V2039,V2040,V2041,V2042,V2043,V2044,V2045,V2046,V2047,V2048
3,1.794234,-0.722560,-0.611274,0.246038,1.952007,-0.687019,1.919408,-0.536251,0.228596,0.580031,...,-0.649727,-0.472004,-0.308215,0.516337,-0.570775,1.686378,-0.368707,0.705388,1.560745,-0.235392
4,-0.614232,-0.723645,-0.625850,-0.700769,0.890061,-0.708099,-0.517712,0.295749,-0.601133,-0.700777,...,-0.615763,-0.346717,-0.295181,1.710085,-0.606019,6.268199,-0.540629,0.191284,-0.658648,-0.508116
5,-0.340294,2.308569,-0.227442,0.966067,-0.129732,-0.697859,-0.226779,-0.548416,-0.312894,-0.400656,...,-0.645283,-0.061791,-0.307888,-0.174181,-0.576878,-0.374934,-0.537209,-0.327834,-0.034123,-0.432091
6,-0.614547,-0.485109,-0.086185,-0.683730,-0.459531,2.083311,-0.519310,-0.536590,0.068543,-0.705624,...,-0.647847,1.954289,-0.276297,-0.490668,-0.414827,-0.417010,0.753108,-0.539450,-0.652012,-0.483409
7,-0.615582,-0.722336,-0.634885,-0.711567,0.079766,0.923661,-0.523824,-0.547914,2.401222,0.775926,...,-0.313758,0.985131,-0.308478,-0.499445,0.463177,0.013233,-0.334262,-0.544675,-0.653028,-0.465167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10849,-0.615484,-0.636381,-0.603358,0.167440,-0.128089,0.777650,1.705667,-0.540217,-0.022750,-0.647462,...,-0.640117,-0.074280,-0.307193,-0.456712,-0.605985,-0.462110,0.586493,-0.140242,-0.543110,-0.518298
10851,-0.060267,-0.705025,-0.602377,1.079070,-0.534121,0.429713,-0.484970,-0.075476,-0.600968,-0.458346,...,-0.318451,3.351063,-0.252847,-0.432449,-0.605439,0.105908,-0.541663,-0.566663,-0.491385,-0.509674
10852,0.120538,-0.715838,1.009706,-0.712069,-0.397888,0.062870,-0.524864,-0.241348,-0.563695,-0.589039,...,0.468343,-0.392630,-0.308413,-0.499445,-0.246427,-0.474798,1.950026,-0.548894,-0.654915,-0.468898
10853,0.027334,0.058712,-0.636646,-0.721406,-0.617998,0.855609,-0.525910,-0.030632,0.414202,-0.702459,...,0.221599,-0.203220,-0.295472,-0.499440,-0.551983,1.395902,-0.194116,-0.378474,-0.576358,-0.517112


In [ ]:
fit = glmnet(y=faces[[i]][train],x=vgg[train,],nfolds = cv_folds, alpha=1,family="binomial",standardize = T,intercept = T)